# Document a time series forecasting model

Use the [FRED](https://fred.stlouisfed.org/) sample dataset to train a simple time series model and document that model with the ValidMind Developer Framework.

As part of the notebook, you will learn how to train a simple model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train a simple time series model
- Running a ValidMind test suite to quickly generate documentation about the data and model

<a id='toc0_'></a>

## Contents  

<a id='toc1_'></a>

## About ValidMind

ValidMind is a platform for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?

If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

<div class="alert alert-block alert-info" style="background-color: #f7e4ee; color: black; border: 1px solid black;">For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — <a href="https://app.prod.validmind.ai"><b>Sign up now</b></a></div>

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Time Series Forecasting** as the template and **Credit Risk - Underwriting - Loan** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
# Replace with your code snippet
import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline

<a id='toc4_1_'></a>

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
from validmind.datasets.regression import fred_timeseries 

target_column = fred_timeseries.target_column

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{target_column}'"
)

raw_df = fred_timeseries.load_data()
raw_df.head()

<a id='toc6_'></a>

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.


<a id='toc6_1_'></a>

### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:
- TBC
- TBC

In [ ]:
# Split the original dataset into training and test sets for the primary model
train_df, test_df = train_test_split(raw_df, test_size=0.2, shuffle=False)
# Remove the first row from the train set to align with the differenced data
train_df = train_df.iloc[1:]

# Prepare the training and test sets for the baseline model
train_baseline_df = train_df[[target_column, "FEDFUNDS"]]
test_baseline_df = test_df[[target_column, "FEDFUNDS"]]

#  Take the first different across all variables
diff_df = raw_df.diff().dropna()

# Use all features for the primary model
train_diff_df, test_diff_df = train_test_split(diff_df, test_size=0.2, shuffle=False)

# Baseline model with only "FEDFUNDS"
baseline_features = [target_column, "FEDFUNDS"]
train_baseline_diff_df = train_diff_df[baseline_features]
test_baseline_diff_df = test_diff_df[baseline_features]

# Primary model with different features
primary_features = train_diff_df.columns.tolist()
train_primary_diff_df = train_diff_df[primary_features]
test_primary_diff_df = test_diff_df[primary_features]

### Train baseline and primary models

In [ ]:
def fit_model(train_df, target_column):
    """
    Trains a Random Forest Regressor model.
    
    Parameters:
    - train_df: DataFrame containing the training data.
    - target_column: Name of the target column.
    
    Returns:
    - model: Trained Random Forest Regressor model.
    """
    X_train = train_df.drop(target_column, axis=1)
    y_train = train_df[target_column]
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_model(model, df, target_column, original_df):
    """
    Makes predictions using the trained model and converts differenced values back to the original scale.
    
    Parameters:
    - model: Trained Random Forest Regressor model.
    - df: DataFrame to make predictions on.
    - target_column: Name of the target column.
    - original_df: Original DataFrame for reference to align previous values.
    
    Returns:
    - y_levels: Actual values in their original scale.
    - y_pred: Predicted values in their original scale.
    """
    X = df.drop(target_column, axis=1)
    y_pred_diff = model.predict(X)
    
    # Align previous values correctly
    previous_values = original_df[target_column].shift(1).reindex(X.index)
    # If there are any NaNs due to shifting, fill them appropriately
    previous_values.fillna(method='bfill', inplace=True)
    
    y_pred = y_pred_diff + previous_values.values
    y_levels = df[target_column] + previous_values.values
    
    return y_levels, y_pred

def evaluate_model(y_true, y_pred):
    """
    Evaluates the model's performance using Mean Squared Error (MSE) and R-squared (R2) score.
    
    Parameters:
    - y_true: Actual values.
    - y_pred: Predicted values.
    
    Returns:
    - mse: Mean Squared Error.
    - r2: R-squared score.
    """
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, r2

In [ ]:
# Fit the baseline model
baseline_model = fit_model(train_baseline_diff_df, target_column)

# Predict and evaluate the baseline model on training data
train_levels_baseline, train_pred_baseline = predict_model(baseline_model, train_baseline_diff_df, target_column, raw_df)
mse_train_baseline, r2_train_baseline = evaluate_model(train_levels_baseline, train_pred_baseline)

# Predict and evaluate the baseline model on test data
test_levels_baseline, test_pred_baseline = predict_model(baseline_model, test_baseline_diff_df, target_column, raw_df)
mse_test_baseline, r2_test_baseline = evaluate_model(test_levels_baseline, test_pred_baseline)

print(f"Baseline Model - Train Mean Squared Error: {mse_train_baseline}")
print(f"Baseline Model - Train R-Squared: {r2_train_baseline}")
print(f"Baseline Model - Test Mean Squared Error: {mse_test_baseline}")
print(f"Baseline Model - Test R-Squared: {r2_test_baseline}")

In [ ]:
# Fit the primary model
primary_model = fit_model(train_primary_diff_df, target_column)

# Predict and evaluate the primary model on training data
train_levels_primary, train_pred_primary = predict_model(primary_model, train_primary_diff_df, target_column, raw_df)
mse_train_primary, r2_train_primary = evaluate_model(train_levels_primary, train_pred_primary)

# Predict and evaluate the primary model on test data
test_levels_primary, test_pred_primary = predict_model(primary_model, test_primary_diff_df, target_column, raw_df)
mse_test_primary, r2_test_primary = evaluate_model(test_levels_primary, test_pred_primary)

print(f"Primary Model - Train Mean Squared Error: {mse_train_primary}")
print(f"Primary Model - Train R-Squared: {r2_train_primary}")
print(f"Primary Model - Test Mean Squared Error: {mse_test_primary}")
print(f"Primary Model - Test R-Squared: {r2_test_primary}")

<a id='toc6_2_'></a>

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_ds = vm.init_dataset(
    input_id="raw_ds",
    dataset=raw_df,
    target_column=target_column,
)

vm_preprocessed_ds = vm.init_dataset(
    input_id="preprocessed_ds",
    dataset=diff_df,
    target_column=target_column,
)

vm_train_baseline_ds = vm.init_dataset(
    input_id="train_baseline_ds",
    dataset=train_baseline_df,
    target_column=target_column,
)

vm_test_baseline_ds = vm.init_dataset(
    input_id="test_baseline_ds",
    dataset=test_baseline_df,
    target_column=target_column,
)

vm_train_ds = vm.init_dataset(
    input_id="train_ds",
    dataset=train_df,
    target_column=target_column,
)

vm_test_ds = vm.init_dataset(
    input_id="test_ds",
    dataset=test_df,
    target_column=target_column,
)

<a id='toc6_3_'></a>

### Initialize the model objects

Additionally, you need to initialize a ValidMind model object (`vm_model`) for each model, that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_baseline_model = vm.init_model(
    baseline_model,
    input_id="baseline_model",
)

vm_primary_model = vm.init_model(
    primary_model,
    input_id="primary_model",
)

<a id='toc6_4_'></a>

### Assign predictions to the datasets

We can now use the assign_predictions() method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [ ]:
vm_train_baseline_ds.assign_predictions(
    model=vm_baseline_model,
    prediction_values=train_pred_baseline,
)

vm_test_baseline_ds.assign_predictions(
    model=vm_baseline_model,
    prediction_values=test_pred_baseline,
)

vm_train_ds.assign_predictions(
    model=vm_primary_model,
    prediction_values=train_pred_primary,
)

vm_test_ds.assign_predictions(
    model=vm_primary_model,
    prediction_values=test_pred_primary,
)

<a id='toc6_5_'></a>

### Run the full suite of tests

This is where it all comes together: you are now ready to run the documentation tests for the model as defined by the documentation template you looked at earlier.

The [`vm.run_documentation_tests`](https://docs.validmind.ai/validmind/validmind.html#run_documentation_tests) function finds and runs every test specified in the template and then uploads all the documentation and test artifacts that get generated to the ValidMind AI Risk Platform.

The function requires information about the inputs to use on every test. These inputs can be passed as an `inputs` argument if we want to use the same inputs for all tests. It's also possible to pass a `config` argument that has information about the `params` and `inputs` that each test requires. The `config` parameter is a dictionary with the following structure:

```python
config = {
    "<test-id>": {
        "params": {
            "param1": "value1",
            "param2": "value2",
            ...
        },
        "inputs": {
            "input1": "value1",
            "input2": "value2",
            ...
        }
    },
    ...
}
```

Each `<test-id>` above corresponds to the test driven block identifiers shown by `vm.preview_template()`. For this model, we will use the default parameters for all tests, but we'll need to specify the input configuration for each one. The method `get_demo_test_config()` below constructs the default input configuration for our demo.

In [ ]:
from validmind.utils import preview_test_config

test_config = fred_timeseries.get_demo_test_config()
preview_test_config(test_config)

Now we can pass the input configuration to `vm.run_documentation_tests()` and run the full suite of tests. The variable `full_suite` then holds the result of these tests.

In [ ]:
full_suite = vm.run_documentation_tests(config=test_config)

<a id='toc7_'></a>

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way — use the ValidMind platform to work with your model documentation.

<a id='toc7_1_'></a>

### Work with your model documentation

1. From the [**Model Inventory**](https://app.prod.validmind.ai/model-inventory) in the ValidMind Platform UI, go to the model you registered earlier.

2. Click and expand the **Model Development** section.

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/working-with-model-documentation.html)

<a id='toc7_2_'></a>

### Discover more learning resources

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.